## Design

High level api i Python som generer Vega-lite spesifikasjon i JSON-format som compiler til en Vega-spesifikasjon. Dette kan bli compilet til HTML-fil som kan bli renderet av frontend. 

Det har en low-level API med et hierarki av python-objekter som korresponderer én-til-én med objekter i "scheme" (..) i vega-lite. Koden for disse er generert automatisk. Dessuten er det en high-level API som gjør at vi kan konstruere spesifikasjoner på en mindre verbos måte.

Vega-lite (og dermed Altair) er bygget på *grammar of graphics* og utvider dette med en interaksjonsgrammatikk. 

Altair er *declarative* i betydningen at hvis spesifiserer *hva* vi ønsker å oppnå, men ikke detaljene om *hvordan* det skal gjøres. Maskinen må riktignok vite *hva* den skal gjøre; denne ambiguiteten blir håndtert gjennom smarte regler for valg av defaults. Dette kan stort sett overkjøres ved behov. 

### Charts

Det grunnleggende objektet. Hver chart er assosiert med ett datasett og én mark.

#### Data

Må være tidy; hver rekke korresponderer med én observasjon og verdi i kolonne angir egenskap til observasjonene langs en dimensjon.

Output fra altair er en spesifikasjon/instruksjon for mapping av data til grafisk element; ikke en figur representert ved bitmap eller noe vector-format. For at det skal være gyldig må spesifikasjon innehold informasjon om dataene. En mulighet er å embedde data i selve spesifikasjonen. Dette gjør spesifikasjonen mindre ryddig og det skalerer dårlig. Det er bedre å kun inkludere adresse til hvor data befinner seg (enten url eller på lokalt filsystem). Nedsiden med denne tilnærmingen er at det forutsetter at adressen må fortsette å være korrekt for at det skal være mulig for frontend å generere figuren. 

##### Data Transformers

Det er noe greier med hvordan data blir behandlet før det inngår i chart... har ikke noe med transform_* ... kan for eksempel bruke
```python
alt.data_transformers.enable()
```
for å automatisk lagre json-fil av data på disk å peke på den i spesifikasjonen. Har ikke fått dette til å fungere og synes uansett det er lite ryddig. Siden jeg så langt er tvunget til å embedde data, så er dirty løsning
```python
alt.data_transformers.disable_max_rows()
```

#### Compound charts

Litt usikker på terminologi. Det er noe top-level greirer. Også litt ukomofortabel med at det ikke er eksplisitt grid; føler jeg mangler litt kontroll på layout, men default ser stort sett greit ut.



```python
resolve_scale()
resolve_axis()
resolve_legend()
```

##### Layer

Legge charts oppå hverandre. Har to måter

```python
alt.layer(chart1, chart2)
chart1 + chart
```
Hvis vi f.eks. vil ha samme encoding men to forskjellige markers (linje og punkt), så kan vi utnytte fleksibilitet i rekkefølger metoder blir callet; vi kan lage encoding uten å spesifisere mark (som i seg selv gir ugyldig spesifikasjon) og deretter sette på marks til dette base plottet
```python
base = alt.Chart(data).encode(x,y)
base.mark_line() + base.mark_point()
```
Kan ha to ulike scales i y-retning
```python
alt.layer(line1, line2).resolve_scale(y='independent')
```

##### Concatination

Kan lage figur med flere subplots(?) ved å enten legge charts oppå eller ved siden av hverandre

```python
# ved siden
alt.hconcat(chart1, chart2)
chart1 | chart2

# oppå
alt.vconcat(chart1, chart2)
chart1 & chart2
```

##### Facets

Dupliserer samme plott på subset derfinert av unike verdier på en kolonne. Spesifiserer det gjennom encoding,
```python
chart.encode(
    columns='fieldname1',
    rows='fieldname2'
).properties(
    height, width # må spesifisere for at samlet figur ikke skal bli for stor. 
)    
```
Litt usikker på om jeg kan ha én field og spre det utover både rows og columns.. Neida, går fint det.

```python
chart.encode(
    facet=alt.Facets('fieldname', columns=2)
)    
```

##### Repeat

Kan bruke repeat-funksjonen for å lage parvis scatterplot..
```python
fields = ['petalLength', 'petalWidth', 'sepalLength', 'sepalWidth']

alt.Chart(iris).mark_point().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='species'
).properties(
    width=150,
    height=150
).repeat(
    row=fields,
    column=fields
)
```

### Marks

Tilsvarer geom i ggplot2, de ulike geometriske objektene i plottet.

#### punkter

```python
mark_point()
mark_circle()
mark_square()
```

Kan bruke ticks til å gi visuell representasjon av empirisk fordeling... kan kombinere med kde eller annet.
```python
mark_tick()
```

#### Rektangler

Kan lage barer for å gi bedre representering av punkt i (x,y)-diagram. Brukes gjerne når ene kolonnen er kategorisk slik at posisjon ikke har numerisk tolkning. I praksis er det ofte bedre å ha horisontale barer siden det er enklere å lese horisontal labels. Hvis vi vil vise flere verdier for hver verdi av den kategoriske kolonnen så vil barene i utgangspunktet overlappe hverandre. Kan sette de ved siden av hverandre eller oppå hverandre. Hvis jeg setter oppå hverandre og normaliserer høyde til 1 så vil hver bar utgjøre andel.
```python
chart.mark_bar().encode(
    y='colname:N',
    x=alt.X('*:Q', aggregate='count', stack='normalize', sort='descending'), #sortere hvis ulike kategorier er ordinal
    color='colname:N'
)
```

Andre ganger vil vi ha rektangler med gitt størrelse... og bruke farge til å angi verdi. Kan også bruke det til å fargelegge bakgrunn for å indikere hendelse i lineplot

```python
mark_rect()
```

#### Linjer

```python
mark_line(
    color, # definerer groupby
    detail, # hvis vi vil gruppere etter annen variabel og få flere linjer med samme farge
)
```

Litt hacky måte å få finere legend i lineplot er å legge på mark_point() med opacity=alt.value(0)

Area plot er litt sånn som linje plots, bare at farglegger under. Da må y-akse begynne på 0. 
```python
mark_area().encode(
    x, 
    alt.Y('count()', stack='normalize') # normalisere slik at høyde = 1 og gir andel for hver delkategori
    color 
)
```

#### Tekst

Kan ha tekst assosierte med de ulike markene i figuren. Da vil vi gjerne at plassering til tekst skal ha sammenheng med plassering til mark og vi bruker samme encoding av x og y kanalene. 

```python
mark_text().encode(
    x,
    y,
    text=alt.Text(tekst, format='.2f') # format følger d3
).configure_text(
    stroke='white', # kan gi det bakgrunn slik at tekst ikke overlapper med andre objekt
    strokeWidth,
    align='left',
    dx, dy,
)    
```
kan kombineres med annen Chart og legge opp for å f.eks. få tall på toppen av bar plot

En alternativ bruk av tekst er å ha den på fast plassering og at verdi avhenger av data. Brukter da alt.value() til å spesifisere plasseringen og må konstruere text i label gjennom transfomeringer

```python
mark_text().encode(
    x=alt.value(20), # offset fra øverste venstre hjørne i antall pixler
    y=alt.value(20),
    text='label:N' # label som vi eksplisitt konstruerer nedenfor
).transform_aggregate( # bruker aggregate til å få noe sammendragsmål
    num_obs = "count(z)"
).transform_calculate( # bruker calculate til å eksplisitt konstuere stringen
    label = "'Antall: ' + datum.num_obs
)    
```

#### Annet

```python
mark_boxplot()
```
Horisontal eller vertikal linje som spanner hele figur.
```python
mark_rule()
```

### Encoding

Spesifiserer mapping fra variabel til Marks ulike estetiske egenskaper

Tolker scale (innhold i mapping mellom dataspace og estetikkspace) ut fra datatype
1. Quantitative (Q)
2. Ordinal (O)
3. Nominal (N)
4. Tid (T)

Skal sjå litt på ulike typer encodings

#### Spesifisere encoding

Det er fleksibel api der vi kan spesifisere på ulike måter. En mulighet er å bruke shorthand, som er en string som angir field (kolonnenavn) og eventuelt datatype og aggregate. Tre ulike måter å spesifisere samme encoding:
```python
chart.encode(
    x='mean(colname):Q')

chart.encode(
    alt.X(shorthand='mean(colname):Q'))

chart.encode(
    alt.X(field='colname',aggregate='mean', type='quantitative))

```
de ulike funksjonene har flere argument enn de som kan angis gjennom shorthand, så får mer fleksibilitet gjennom å eksplisitt bruke funksjonene

#### Ulike typer encoding

Ulike kanaler for å mappe mellom data og mark. Hver har et `scale` argument der vi kan spesifiserer detaljer ved mapping.

##### Posisjon

Hvordan jeg ønsker å håndtere dette avhenger litt av om direkte tolkning av plassering som numerisk verdi eller kun ordinal/nominal distinksjon mellom objekt

```python
chart.encode(
    alt.X('column',
        scale=alt.Scale(domain=(5, 20), # kan sette axis limits
                        zero=False, # inferrer domain (med litt padding) uten begrensing om at den må starte på 0
                        clamp=True)), # observasjon som utenfor view blir "clampet" til margen
        axis=alt.Axis(title='..',
                      titleFontSize=14,
                      format='%Y', # har verktøy for formatering av datetime kolonne
                      format='%', # ganger verdi på akse med 100 og legger til % slik at 0.01 -> 1%
                      labelAngle=0, # kan justere vinkel til tick labels
                      ticks=False, # kan være aktuelt å fjerne ticks for en del typer plot 
                      tickCount, # én måte å kontrollere ticks (og dermed grid)
                      
                      ) 
        sort=alt.EncodingSortField(field='other_col', # sortere etter numerisk verdi til annen kolonne
                                   order='descending',
                                   op) # aggregeringsfunksjon som er nødvendig hvis flere verdier for hver encoding..
)          
```

```python
alt.Axis(
    title, # axis label
    labels,
    ticks,
    format='%-d/%-m/%Y' # kontrollere hvis tid
)
```

##### Farge

Får mer kontroll over fargevalg ved å kombinere alt.Color() med et Scale-objekt

Trenger litt oversikt over schemes og built-in farger.. hvis man bruker vanilje blå, rød osv. så er det kjempestygt. 
```python

custom_scale = alt.Scale(domain=['val0', 'val1',...], range=['color0', 'color1',...])
builtin_scale = alt.Scale(scheme='dark2)

chart.encode(
    alt.Color('column:N',
              scale=alt.Scale(scheme, # bruke built-in mapping
                              domain, # spesifiere custom gjennom domain og range
                              range, # domain=[val1, val2], range=['color1', 'color2']. 
              sort, # liste av verdier i column eller alt.EncodingSortField(..), bruke til å kontrollere legend,
              legend, # farge vil dukke opp i legend, konfiguerer her.
             )
```

##### Size

```python
chart.encode(
    alt.size('column', 
             scale=alt.Scale(range([a,b]) # justere hvor store og små markene kan blir
)    
```

##### Opacity

Kan justere opacity hvis mange overlappene objekt, f.eks. i scatterplot eller lineplot. Har sett at vi kan fremheve noen punkter ved å gjør farge til alle grå. Alternativ måte er å justere opacity. Kan f.eks. ha mange svake linjer, en sterk linje som viser gjennomsnittet og så kan man fremheve ulike linjer ved å holde musen over.
```python
hover = alt.selection_single(on='mouseover', nearest=True, fields=['state'], empty='none')
chart.mark_line().encode(
    x,y,
    opacity=alt.condition(hover, alt.value(1.0), alt.value(0.1))
)    
```

##### Order

Kan spesifiserer rekkefølge... for eksempel i stacked bar chart.. litt usikker på hvordan jeg bruker i praksis. vil bruke sort=alt.EncodingSortField() for å sortere rekkefølge på samme måte som jeg sortere farger, men får det ikke til.

Kan også bruke til å bestemme rekkefølge i scatterplot med ulik størrelse, slik at større sirkler blir tegnet først og jeg kan få tilgang til mindre sirkler med mus for å få ut tooltip ..
```python
chart.encode(
    x,y,color='colname',
    order=alt.Order('colname',sort='descending') # ... 
)    
```

##### Shape


##### Tooltip

Bestemme hvilken tekst som vises når musen hoverer over markør.
```python
chart.encode(
    tooltip = [col_names], # fungerer, men vet ikke hvordan jeg får annen tittel enn colname label
    tooltip = alt.Tooltip()
)
```

### Transformasjoner

Har mye verktøy for å gjøre transformasjoner/aggregeringer internt. Om mulig så er det ofte greiere å gjøre dette i pandas, men kan være at jeg trenger ulike views av data for sammensatte figurer eller at jeg jobber direkte med data gjennom url.

Merk uansett at den kan ta begrenset med data (default øvre grense er 5000), så for mange datasett har jeg ikke annet valg enn å aggregere/filtrere ex-ante.


Kan spesifiserer transformasjon av variabel,
```python
chart.encode(
    attribute='mean(colname)'
)
```
Kan også bruke
```python
chart.encode(
    x='colname',
    y='count()
)
```
for at plassering i y-retning avhenger av antall observasjoner som encodet på gitt plassering i x-retning

#### filter

```python
chart.transform_filter(
    selector1 | selector2, # kan ha logisk condition med enten union eller snitt av selectors..
    
)
```

#### aggregate

Har i hvert fall tre måter jeg kan gjøre aggegering på:

1. Kan gjøre eksplisitt med transform_ funksjon slik at jeg få nye field i mitt data. 

```python
chart.transform_aggregate(
    agg_field_name='mean(colname2)', # alt.AggregatedFieldDef, må ha `op` (aggfunksjon), fieldname (arg i `op`) og `as` 
    groupby=['colname'] # må være array
).encode(
    x='colname',
    y='agg_field_name' # eks 'mean_of_colname2'
)
```

2.  Kan eventuelt gjøre det gjennom encoding i stedet for å transformere ex-post

```python
chart.encode(
    x='colname',
    y='mean(colname2)' # skjønner at groupby colname .. kan eventuelle kombinere med binning hvis colname er kvantiativ
)
```


3. Gjøre med eksplisitt objekt i encoding i stedet for bare shorthand

```python
chart.encode(
    x='colname',
    y=alt.Y('colname2:', aggregate='mean')
)    
```

#### binning

Aggegere med utgangspunkt i delmengder (intervaller) av kontinuerlig variabel. Blir en måte å konstruere slags diskret variabel som vi deretter kan kjøre groupby på og finne aggegert mål på hver gruppe. Kjører altså binning og grouping i ett. Vanlig å bruke `count()` for å lage histogram, men ingenting i veien for å bruke andre aggregeringsfunksjoner

Kan lage hist ved å telle antall observasjon i hver bin, men kan også lage andre ting.

##### Histogram

Kan enten spesifisere at bin=True eller initialisere Bin-objekt som gir oss flere arguemnt
```python
chart.encode(
    x=alt.X('Miles_per_Gallon', bin=alt.Bin(maxbins=20)),
    y='count()'
)
```
Bin har litt ulike argumenter, men i praksis vil bare bruke maxbins siden den prøver å utlede en god partisjonering (splitte på heltall).

Hvis jeg vil bruke gruppene i flere chart komponenter så kan jeg eksplisitt lage ny kolonne som inneholder binning (som jeg deretter kan referere til ved navn på andre komponenter,
```python
chart.encode(
    x='x_binned:O',
    y='count()'
).transform_bin('x_binned', field=x)    
```

Kan også lage histogram der vi bruker bruker farge til å indikere verdi i hver bin i stedet for lengde på bar i y-retning.
```python
alt.Chart(data).mark_rect().encode(
    alt.X('col_x', bin=True),
    alt.Y('col_y', bin=True),
    color='count():O'
)
```

##### Binning med tid

Har ulike funksjoner for å binne data med tidsdimensjon. Aggregert mål på observasjoner innenfor diskret tidsintervall.
```python
chart.encode(
    x='day(time_var):O', # alternativer: hours, month, year (mer? sikkert minutter og sekunder..)
    y='mean(some_var)'
)
```

#### density

Vi kan alternativt bruker kde til å estimere tettheten til fordeling

```python
chart.mark_area().transform_density(
    density='col',
    as_= ['col', 'tetthet']
).encode(
    x='col:Q',
    y='tetthet:Q'
)
```

#### calculate

Kan ha lyst til å konstruere nye kolonner fra funksjon på de eksisterende. Kan bruke stringfunksjon som blir parset, eller være mer eksplisitt og pythonisk med funksjoner fra alt.expr modulen
```python
chart.transform_calculate(
    new_col = "func(datum.old_col)", # må bruke datum. for å få tilgang til kolonne
    new_col = alt.expr.max(datum.old_col) # finnes matematiske funksjoner, ting som er definert på arrays mm.
)    
```

#### pivot

Omforme fra long til wide. Må spesfisiere
1. index (hvilken kolonne som identifiserer unike observasjoner)
2. column (hvilke kolonne hvorav de unike verdiene vil være navn på kolonne i wide-format)
3. value (hvilken kolonne der verdiene fyller opp (index, column)-par.

```python
alt.Chart(df).transform_pivot(
    'type', # unike verdier herfra gir nye field names
    groupby=['country'], # unike verdier herfra er index
    value='count'
)
```

#### timeunit

Kan trekke ut egenskap fra tids-kolonne og gruppere alle observasjoner som deler den egenskaper. Kan være adskilt i tid.. hvis grupperer på egenskaper som vokser monotont. Ellers kan vi dele inn etter egenskap som er syklisk (dag, måned) og få annen type gruppering.

```python
chart.transform_timeunit(
    month="month(time)"
).encode(
    x = 'month:O',
    y = 'mean(temp):Q'
)    
```

### Seleksjon

Workflow:
1. Initialisere seleksjonsobjekt (som eventuelt er bundet til slider, meny, buttons eller legend
2. Legge til seleksjonsobjekt til chart med .add_selection()
3. Bruke alt.condition() på attribut i encode-funksjonen for å gjøre ting avhengig av seleksjon
4. Eller filter/scales/domain

#### Typer seleksjon

Har tre typer seleksjonsobjeter

```python
alt.selection_interval(
    encodings, # ['x'] hvis kun intervall på x-akse. Default er ['x','y'] som gir rektangel
    bind, # 'scales' gir basically samme som interact.. kan konstruere zoom gjennom bind på intervall..
    empty, # enten 'all' eller 'none'. om de på default regnes for å være inni seleksjon eller ikke.
    resolve, # default er global slik at all filtrering på én seleksjon, men kan spesifisere union for å få flere ...
    
)    
```

```python
alt.selection_single(
    on, # 'click', 'mouseover'
    nearest, # bør være True hvis on='mouseover'
    encodings, # velge alle marks med samme egenskap av encoding, eks ['color']
    fields, # i stedet for encoding (posisjon/farge) kan vi også velge alle med samme verdi i field direkte..
    init, # {'fieldname':value}
    bind, # kan binde til alt.binding_range(min,max,step) for å få slider..
    
)
```

```python
alt.selection_multi(

)
```

##### Flere bindinger på samme selection-objekt

```python
alt.selection(
    fields=['field1', 'field2'],
    bind={'field1': alt.binding(...), 'field2': alt.bidning(...)}
)    
```

#### Typer binding

I stedet for at seleksjon får input fra mus på selve figuren, kan jeg lage binding-objekt og binde verdi av seleksjonsobjekt til input fra dette objektet.

```python
alt.binding_range(
    
)    
```
```python
alt.binding_select(
    
)
```


```python
alt.binding_radio(
    
)
```
```python
alt.binding_checkbox(
    
)
```

#### Typer av interaktivitet

Skal nå se på måter vi kan endre figuren gjennom verdi til selection objektt

##### Conditional encodings

```python
alt.condition(predicate=selection_obj,
              if_true=mapping, # i praksis kolonnenavn.. hva den mapper fra.
              if_false=some_value) # eller kanskje annen mapping
```

Bruker én mapping fra data til visuell egenskap for observasjon/objekt som er valgt og annen mapping for resten. Bruker condition-funksjon som argument for egenskap i encode, eks:
```python
chart.encode(x,y,
    color=alt.condition(interval, 'species', alt.value('lightgray'))
).add_selection(interval)
```

##### Filter

```python
chart.transform_filter(
    selection_obj
).transform_filter(
    "datum.colname == selection_obj.colname"
)    
```

##### Scales

##### Domain

Kan bruke seleksjon i én chart til å definere domain i annen chart slik at vi i praksis zoomer inn på angitt intervall.

```python
chart = base.encode(x=alt.X('date:T', scale=alt.Scale(domain=interval.ref()))) # litt usikker på hva .ref() gjør
view = base.add_selection(interval)
chart & view                    
```
Har samme x encoding, men på chart er domenet til x angitt av interval som vi bestemmer i view

### Configuration

Har smarte defaults, men det vil ofte være lurt å spikke på plottet. Har tre ulike nivåer vi kan konfiguerer egenskaper til element i chart. Mer lokalt har forkjørsrett. Hvis jeg spesifiserer top-level konfigurering, så kan ikke denne charten inngå i compound. 

Kan spesifisere global i python session gjennom *theme* (analogt til ggplot2)

Har tre nivåer der vi kan spesifisere egenskaper ved utseendet til chart

```python
# 'red'>'blue'>'green' fordi encode>local>global
alt.Chart(data).mark_point(color='blue').encode( 
    x,y,
    color=alt.value('red') # må bruke alt.value for at den ikke skal tro det er kolonnenavn
).configure_point(color='green') 
```
Best practice er å spesifisere på så lavt/lokalt nivå som mulig.

Eksempler på konfiguereringer

```python
alt.Chart(data).mark_point().encode(
    encoding_1='column_1',
    encoding_2=alt.function('column_2', *args)
    ...
).properties(
    height,
    width,
    ...
).configure_axis(
    labelFontSize,
    titleFontSize
).configure_legend(
    orient
).configure_point(
    size
)
```

#### Argumenter

```python
).configure_mark(
    align, # vanligvis vil den sentrere mark på (x,y), men kan spesifisere 'left' eller 'right'. Relevant for tekst
    dx, dy, # hvis vi vil manuelt dytte det litt for å skape offset slik at det ikke overlapper
    baseline, # hm..
    
    
)    
```

#### Properties

Kan sette en del egenskaper gjennom .properties metoden ..

#### Legend

Kontrollerer gjennom encoding..

```python
chart.encode(
    ...,
    alt.Color('colname', legend=alt.Legend(...)) # encoding som generer legend. Spesifiere legend=None hvis supress.
)
```
her er argumentene
```python
alt.Legend(
    title, # None hvis supress
    titleFontSize,
    labelFontSize,
    orient, # 'bottom' hvis jeg vil ha under plottet
    titleOrient, # 'left' hvis legend er under plottet
    symbolStrokeWidth, # må forstørre hvis jeg bruke lineplot
    
)
```

Viktig å kontrollere legend slik at enkelt å mappe til figur. Vil kontrollere rekkefølgen. Litt lite intuitivt, men gjør dette gjennom å sortere farge fordi 
1. det er en rekkefølge av farger i cmap som jeg bruker
2. rekkefølge av labels i legend tilsvarer rekken av farger

så må derfor endre hvilke labels som korresponderer med hvilke farger!

##### Interaktiv legend

Kan lage interaktiv legend ved å binde selection.. litt begrenset funksjonalitet tror jeg.
```python
click = alt.selection_single(encodings=['color'], bind='legend')
chart.encode(
    opacity=alt.condition(click, alt.value(1), alt.value(0.2))
).add_selection(click)
```

### Subset filtrering histogram

Betinget histogram for andre fordelinger for angitt subset av en fordeling. Består av to layers: 1. vanilje hist med brush 2. hist filtrert på observasjon med annen farge. Repeater dette for alle kolonner vi er interessert i.

Vil ha to forbedringer: vil at seleksjon skal ta hele barer i histogram og vil ha har grid med både rekke og kolonne..
```python
brush = alt.selection_interval(encodings=['x'], empty='none')
base = alt.Chart(cars).mark_bar().encode(
    alt.X(alt.repeat('column'), type='quantitative', bin=alt.Bin(maxbins=20)),
    alt.Y('count()')
).properties(
    width=300, height=200
)

base_background = base.properties(selection=brush)
base_highlight = base.encode(
    color=alt.value('goldenrod')
).transform_filter(
    brush.ref()
)

alt.layer(base_background, base_highlight).repeat(column=['Miles_per_Gallon', 'Displacement'])
```

### Subset filtrering scatter

```python
brush = alt.selection_interval(encodings=['x', 'y'], empty='none')
base = alt.Chart(cars).mark_point().encode(
    color=alt.condition(brush, alt.value('red'), alt.value('lightblue'))
).add_selection(brush)

points1 = base.encode(alt.X('Miles_per_Gallon'), alt.Y('Horsepower'))
points2 = base.encode(x='Weight_in_lbs', y='Displacement')
alt.hconcat(points1, points2)
```

## Vega-lite

Altair genererer en vega-lite-spesifikasjon i json-format som compiler til vega-spesifikasjon. Det er altså strengt tatt bare en api i Python, og den underliggende strukturen kommer fra vega-lite som det derfor er greit å sette seg inn i.

### Grafisk grammatikk

Enhetsplott består av tuppelen (data, transformations, mark-type, encodings, seleksjon), der
- data er tabulær og hver rekke angir verdier til én observasjon langs navngitte egenskaper (såkalt tidy data)
- transformasjon kan være enhetstransformasjon, men også blant annet filtrering, binning og aggregering
- mark-type er geometrisk objekt; kun én type per enhetsplot
- encoding spesifiserer mapping fra data til visuelle kanaler (estetiske egenskaper) ved mark. Kan være flere encodings på samme mark fordi den har flere kanaler.
- seleksjon kan være enhetseleksjon, men kan også åpne for interaktivitet. se under.

Mer formelt kan vi si hver encoding består av tuppelen (channel, field, data-type, value, function, scale, guide), der
- channel er visuelle egenskaper som plassering, farge, størrelse og form, samt rekkefølge (order)..
- field er navn på kolonne i data. 
- data-type er nominal, ordinal, kvantiativ eller temporal
- value vil som default være verdier fra kolonnen som er angitt i field, men kan også bruke konstant verdier
- function ... er jeg litt usikker på.
- scale spesifiserer den eksakte mappingen mellom data-space og visuell-space... Eks hvilken verdi som korresponderer med hvilken farge.
- guide beskriver invers av transformasjon så vi kan mappe tilbake fra visuell til data, gjennom legends og sånn.

Merk at vega-lite (og altair) automatisk genererer scale og guide ut fra channel og data-type i henhold til best practices (ordinal vs kontinuerlig fargeskala mm).

### Algebra for komposisjoner

Gjør poeng ut av at det er er view algebra i stedet for table algebra ? uansett, så håndterer det alignment, valg av scale og unngår unødvendig duplisering av guide (legend) automatisk. Har fire operasjoner
1. Layer, hver argument må være enhetsplot i motsetning til andre som bare trenger view... hm.
2. Concat
3. Facet, samme plot på subset av data
4. Repeeat, hele data på hver

Har resolve-egenskap som sier hvordan det skal håndtere scales..

### Interaksjonsgrammatikk

Seleksjon består av tuppelen (name, type, predicate, domain eller range, event, init, transformation, resolve), der innhold på hver enkelt blir litt low-level for meg foreløbig. Har tre ulike typer
1. Single point
2. List of points
3. Interval

Finnes også transformasjoner av seleksjonsobjekt... manipulere innhold.

## Eksempler

Tekst annotation på multiline graf. 
```python
from vega_datasets import data
cars = data.cars()

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Year'], empty='none')

line = alt.Chart(cars).mark_line().encode(
    x='Year:T',
    y='mean(Horsepower):Q',
    color='Origin'
)

points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
).add_selection(nearest)

rule = alt.Chart(cars).transform_filter(nearest).mark_rule().encode(
    x='Year:T',
    color=alt.value('gray')
)

text = line.mark_text(strokeWidth=6,align='left', dx=5, dy=-10).encode(
    text=alt.condition(nearest, 'mean(Horsepower):Q' ,alt.value(' '))
)

line + points + rule + text
```

Alternativ til tekst annotation er å lage multiline tooltip. Må passe på at rekkefølge samsvarer med rekkefølge i legend.
```python
source = data.stocks()
selection = alt.selection_single(
    fields=['date'], nearest=True, on='mouseover', empty='none', clear='mouseout')

base = alt.Chart(source).encode(x='date:T')

lines = base.mark_line().encode(
    y='price',
    color='symbol'
)

points = lines.mark_point().transform_filter(selection) # kun visuell, kunne brukt opacity

rule = base.transform_pivot(
    'symbol', groupby=['date'], value='price'
).mark_rule().encode(
    opacity=alt.condition(selection, alt.value(0.3), alt.value(0)),
    tooltip=[alt.Tooltip(c, type='quantitative') for c in sorted(source.symbol)]
).add_selection(selection)

lines + points + rule
```